# Terahertz (THz) Imaging Reconstruction Demo
This notebook demonstrates a basic reconstruction pipeline for Terahertz Imaging using placeholder operators and reconstructors. The `TerahertzOperator` currently uses a generic system matrix (randomly initialized if not provided) and would need to be replaced with a system matrix or model appropriate for a specific THz imaging setup (e.g., THz CT, THz pulsed imaging).

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

# Adjust path to import from reconlib 
import sys
# sys.path.append('../../../') # Adjust as needed

from reconlib.modalities.terahertz.operators import TerahertzOperator
from reconlib.modalities.terahertz.reconstructors import tv_reconstruction_thz
from reconlib.modalities.terahertz.utils import generate_thz_phantom, plot_thz_results

print(f"PyTorch version: {torch.__version__}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 1. Setup Parameters and Phantom

In [ ]:
image_shape_thz = (64, 64)  # (Ny, Nx)
num_pixels_thz = np.prod(image_shape_thz)

# For a system matrix based operator, define number of measurements
# This could be, e.g., number of views * number of detectors in THz CT
# Or number of k-space samples in a Fourier-based THz system.
num_measurements_thz = num_pixels_thz // 2 # Example: undersampled system

# Generate a simple phantom (e.g., representing absorption or refractive index variations)
true_thz_image = generate_thz_phantom(image_shape_thz, num_shapes=3, shape_type='rect', device=device)

plt.figure(figsize=(6,6))
plt.imshow(true_thz_image.cpu().numpy(), cmap='magma')
plt.title('True THz Phantom Image')
plt.xlabel('X (pixels)')
plt.ylabel('Y (pixels)')
plt.colorbar(label='Property Value')
plt.show()

## 2. Initialize Operator and Simulate Data
For this demo, we'll let the `TerahertzOperator` create its own random placeholder system matrix if one isn't provided. In a real scenario, this matrix `A` would come from the physics of the THz system.

In [ ]:
# Option 1: Let the operator create a random system matrix (for demo purposes only)
# thz_operator = TerahertzOperator(image_shape=image_shape_thz, system_matrix=None, device=device)

# Option 2: Define a specific system matrix (more realistic placeholder)
is_complex_system = False # Set to True for a complex-valued system matrix
dtype_system = torch.complex64 if is_complex_system else torch.float32
thz_system_matrix = torch.randn(num_measurements_thz, num_pixels_thz, dtype=dtype_system, device=device) * 0.1

thz_operator = TerahertzOperator(
    image_shape=image_shape_thz,
    system_matrix=thz_system_matrix,
    device=device
)

# Ensure phantom is of appropriate dtype for the operator 
phantom_for_op = true_thz_image.to(dtype_system) if is_complex_system and not true_thz_image.is_complex() else true_thz_image
if not is_complex_system and phantom_for_op.is_complex():
    phantom_for_op = phantom_for_op.real 

# Simulate THz measurement data using the forward operator y = A * x
y_thz_measurements = thz_operator.op(phantom_for_op)

print(f"Simulated THz measurement data shape: {y_thz_measurements.shape}")

# Visualize measurement data (if 1D)
if y_thz_measurements.ndim == 1:
    plt.figure(figsize=(8,5))
    plt.plot(y_thz_measurements.cpu().numpy() if not y_thz_measurements.is_complex() else y_thz_measurements.abs().cpu().numpy())
    plt.title('Simulated THz Measurements (Placeholder System)')
    plt.xlabel('Measurement Index')
    plt.ylabel('Signal Value')
    plt.show()

## 3. Perform Reconstruction
Using Total Variation (TV) regularization with Proximal Gradient.

In [ ]:
lambda_tv_thz = 0.005      # TV regularization strength
iterations_thz = 30       # Number of proximal gradient iterations (low for demo)
step_size_thz = 0.01     # Step size for proximal gradient
tv_prox_iters_thz = 5     # Iterations for TV prox

# Perform reconstruction
reconstructed_thz_img = tv_reconstruction_thz(
    y_thz_data=y_thz_measurements,
    thz_operator=thz_operator,
    lambda_tv=lambda_tv_thz,
    iterations=iterations_thz,
    step_size=step_size_thz,
    tv_prox_iterations=tv_prox_iters_thz,
    is_3d_tv=len(image_shape_thz)==3,
    verbose=True
)

print(f"Reconstructed THz image shape: {reconstructed_thz_img.shape}")

## 4. Display Results

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

im1 = axes[0].imshow(true_thz_image.cpu().numpy(), cmap='magma')
axes[0].set_title('Ground Truth THz Image')
axes[0].set_xlabel('X (pixels)')
axes[0].set_ylabel('Y (pixels)')
fig.colorbar(im1, ax=axes[0], fraction=0.046, pad=0.04)

img_display = reconstructed_thz_img.cpu().numpy()
if reconstructed_thz_img.is_complex():
    print("Displaying magnitude of complex reconstructed image.")
    img_display = np.abs(img_display)
    
im2 = axes[1].imshow(img_display, cmap='magma')
axes[1].set_title(f'Reconstructed THz Image (TV, {iterations_thz} iters - Placeholder)')
axes[1].set_xlabel('X (pixels)')
axes[1].set_ylabel('Y (pixels)')
fig.colorbar(im2, ax=axes[1], fraction=0.046, pad=0.04)

plt.tight_layout()
plt.show()

# Using the utility plot function (currently a placeholder itself)
plot_thz_results(
    true_image=true_thz_image,
    reconstructed_image=reconstructed_thz_img,
    measurement_data=y_thz_measurements
)

## 5. Further Considerations
To make this notebook fully functional for a specific THz imaging modality:
1. **Define `TerahertzOperator` accurately**: 
    - For THz CT (Computed Tomography): This might involve implementing a Radon transform operator or using a system matrix derived from ray tracing or wave propagation through the object at different angles.
    - For THz Pulsed Imaging / Spectroscopy: The operator might model the convolution with a THz pulse and reflections/transmissions. Deconvolution might be part of the operator or preprocessing.
    - For THz Holography / Diffractive Imaging: The operator would model wave propagation (e.g., using Angular Spectrum Method or Fresnel diffraction) from the object plane to a detector plane.
    Ensure the `op_adj` (adjoint) passes the dot-product test with `op` (forward).
2. **Realistic System Matrix**: If using a system matrix approach, this matrix `A` must be carefully calibrated or simulated based on the experimental setup.
3. **Data Type**: THz data can be complex (amplitude and phase from THz-TDS) or real-valued (intensity from bolometers/pyroelectric cameras). Ensure operator and image types are consistent.
4. **Regularization**: TV is a good start. Other regularizers like L1-wavelet might be suitable depending on the expected sparsity in the THz image or its transform domains.
5. **Noise**: Add realistic noise (e.g., Gaussian, Johnson-Nyquist) to `y_thz_measurements`.